In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gc

import pandas as pd
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.optim import AdamW
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel #AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# sentences = ['Привет! Как твои дела?',
#              'А правда, что 42 твое любимое число?']

# model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

# encoded_input = tokenizer(sentences, padding='max_length', truncation=True, max_length=24, return_tensors='pt')

# with torch.no_grad():
#     model_output = model(**encoded_input)

# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [44]:
company = 'DSKY'
tg = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg_raw.parquet')
comp = tg[tg[company] == True].copy()
comp.drop(columns=['views', 'forwards', 'fwd_from', 'source'], inplace=True)
comp.dropna(inplace=True)
comp = comp[comp['date'] < '2022-01-01'].copy()
cols = ['date', 'message'] + [col for col in comp.columns if f'{company}_' in col]
comp = comp[cols].copy()
train_data, test_data = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
cols = [col for col in comp.columns if f'{company}_' in col]

In [50]:
train_texts, val_texts = train_data['message'].values, test_data['message'].values
train_target, val_target = train_data[cols].values, test_data[cols].values

### Большой Bert

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

In [4]:
class BertClassifier(nn.Module):

    def __init__(self, n_classes=7):

        super(BertClassifier, self).__init__()

        self.bert = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(2560, 1024)
        self.fc3 = nn.Linear(1024, n_classes)

        self.drop1 = nn.Dropout(0.1)
        self.drop2 = nn.Dropout(0.1)

        self.act = nn.ReLU()

    def forward(self, input_id, token_type, mask):
        model_output = self.bert(input_id, token_type, mask)
        sentence_embeddings = mean_pooling(model_output, mask)
        embeddings = torch.cat([model_output[1], sentence_embeddings], dim=1)
        embeddings1 = self.fc1(embeddings)
        embeddings = torch.cat([embeddings, embeddings1], dim=1)
        embeddings = self.act(embeddings)
        embeddings = self.drop1(embeddings)
        embeddings = self.fc2(embeddings)
        embeddings = self.act(embeddings)
        embeddings = self.drop2(embeddings)
        output = self.fc3(embeddings)

        return output

In [5]:
class TgDataset(Dataset):
    def __init__(self, texts, targets):
    
      self.labels = targets.astype(int)
      self.texts = [tokenizer(text, 
                          padding='max_length', max_length = 512, truncation=True,
                          return_tensors="pt") for text in texts]

    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):

        batch_texts = self.texts[idx]
        batch_y = self.labels[idx]

        return batch_texts, batch_y

In [29]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, device, epochs, scheduler=None):

    mean_1_min, mean_5_min, mean_10_min, mean_15_min, mean_30_min, mean_1_hour, mean_1_day = train_target.mean(axis=0)

    model = model.to(device)

    for epoch_num in range(epochs):
        model.train()

        acc_1_min_train = 0
        acc_5_min_train = 0
        acc_10_min_train = 0
        acc_15_min_train = 0
        acc_30_min_train = 0
        acc_1_hour_train = 0
        acc_1_day_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)

            print(train_input.keys())
            input_id = train_input['input_ids'].squeeze(1).to(device)
            token_type = train_input['token_type_ids'].squeeze(1).to(device)
            mask = train_input['attention_mask'].squeeze(1).to(device)

            output = model(input_id, token_type, mask)
            batch_loss = criterion(output, train_label.float())
            total_loss_train += batch_loss.item()
            out = torch.sigmoid(output)

            acc_1_min_train += ((out[:,0] > mean_1_min).int() == train_label[:,0]).sum().item()
            acc_5_min_train += ((out[:,1] > mean_5_min).int() == train_label[:,1]).sum().item()
            acc_10_min_train += ((out[:,2] > mean_10_min).int() == train_label[:,2]).sum().item()
            acc_15_min_train += ((out[:,3] > mean_15_min).int() == train_label[:,3]).sum().item()
            acc_30_min_train += ((out[:,4] > mean_30_min).int() == train_label[:,4]).sum().item()
            acc_1_hour_train += ((out[:,5] > mean_1_hour).int() == train_label[:,5]).sum().item()
            acc_1_day_train += ((out[:,6] > mean_1_day).int() == train_label[:,6]).sum().item()

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()
      
        acc_1_min_val = 0
        acc_5_min_val = 0
        acc_10_min_val = 0
        acc_15_min_val = 0
        acc_30_min_val = 0
        acc_1_hour_val = 0
        acc_1_day_val = 0
        total_loss_val = 0

        with torch.no_grad():
            model.eval()
            for val_input, val_label in tqdm(val_dataloader):

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                token_type = val_input['token_type_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)

                output = model(input_id, token_type, mask)

                batch_loss = criterion(output, val_label.float())
                total_loss_val += batch_loss.item()
                out = torch.sigmoid(output)

                acc_1_min_val += ((out[:,0] > mean_1_min).int() == val_label[:,0]).sum().item()
                acc_5_min_val += ((out[:,1] > mean_5_min).int() == val_label[:,1]).sum().item()
                acc_10_min_val += ((out[:,2] > mean_10_min).int() == val_label[:,2]).sum().item()
                acc_15_min_val += ((out[:,3] > mean_15_min).int() == val_label[:,3]).sum().item()
                acc_30_min_val += ((out[:,4] > mean_30_min).int() == val_label[:,4]).sum().item()
                acc_1_hour_val += ((out[:,5] > mean_1_hour).int() == val_label[:,5]).sum().item()
                acc_1_day_val += ((out[:,6] > mean_1_day).int() == val_label[:,6]).sum().item()
      
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .5f} \
                | Train Acc 1 min: {acc_1_min_train / len(train_dataset): .5f} \
                | Train Acc 5 min: {acc_5_min_train / len(train_dataset): .5f} \
                | Train Acc 10 min: {acc_10_min_train / len(train_dataset): .5f} \
                | Train Acc 15 min: {acc_15_min_train / len(train_dataset): .5f} \
                | Train Acc 30 min: {acc_30_min_train / len(train_dataset): .5f} \
                | Train Acc 1 hour: {acc_1_hour_train / len(train_dataset): .5f} \
                | Train Acc 1 day: {acc_1_day_train / len(train_dataset): .5f} \
                | Val Loss: {total_loss_val / len(val_dataset): .5f} \
                | Val Acc 1 min: {acc_1_min_val / len(val_dataset): .5f} \
                | Val Acc 5 min: {acc_5_min_val / len(val_dataset): .5f} \
                | Val Acc 10 min: {acc_10_min_val / len(val_dataset): .5f} \
                | Val Acc 15 min: {acc_15_min_val / len(val_dataset): .5f} \
                | Val Acc 30 min: {acc_30_min_val / len(val_dataset): .5f} \
                | Val Acc 1 hour: {acc_1_hour_val / len(val_dataset): .5f} \
                | Val Acc 1 day: {acc_1_day_val / len(val_dataset): .5f}')

In [11]:
batch_size = 4
epochs = 3
train_dataset, val_dataset = TgDataset(train_texts, train_target), TgDataset(val_texts, val_target)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
total_steps = len(train_dataloader) * epochs

In [12]:
n_classes = 7
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BertClassifier(n_classes)
LR = 2e-5
optimizer = AdamW(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(0.1*total_steps),
                                            num_training_steps = total_steps)

In [ ]:
# model.bert.requires_grad = False

In [ ]:
# train(model, train_dataloader, val_dataloader, optimizer, criterion, device, 1)

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/Диссертация/tg_sbert_7_cls.pth')

In [ ]:
# model.bert.requires_grad = True

In [14]:
model.load_state_dict(torch.load('/content/drive/MyDrive/tg_sbert_7_cls.pth'))

<All keys matched successfully>

In [15]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, device, epochs, scheduler)

100%|██████████| 111/111 [00:48<00:00,  2.31it/s]


Epochs: 1 | Train Loss:  0.17327                 | Train Acc 1 min:  0.49490                 | Train Acc 5 min:  0.50340                 | Train Acc 10 min:  0.48131                 | Train Acc 15 min:  0.51586                 | Train Acc 30 min:  0.51925                 | Train Acc 1 hour:  0.50566                 | Train Acc 1 day:  0.48471                 | Val Loss:  0.16980                 | Val Acc 1 min:  0.40045                 | Val Acc 5 min:  0.37104                 | Val Acc 10 min:  0.38462                 | Val Acc 15 min:  0.38009                 | Val Acc 30 min:  0.41629                 | Val Acc 1 hour:  0.40498                 | Val Acc 1 day:  0.47285


100%|██████████| 111/111 [00:47<00:00,  2.32it/s]


Epochs: 2 | Train Loss:  0.17318                 | Train Acc 1 min:  0.49094                 | Train Acc 5 min:  0.50113                 | Train Acc 10 min:  0.52265                 | Train Acc 15 min:  0.50849                 | Train Acc 30 min:  0.51472                 | Train Acc 1 hour:  0.50283                 | Train Acc 1 day:  0.51019                 | Val Loss:  0.17378                 | Val Acc 1 min:  0.40045                 | Val Acc 5 min:  0.37104                 | Val Acc 10 min:  0.38462                 | Val Acc 15 min:  0.38009                 | Val Acc 30 min:  0.41629                 | Val Acc 1 hour:  0.40498                 | Val Acc 1 day:  0.47285


100%|██████████| 111/111 [00:47<00:00,  2.31it/s]

Epochs: 3 | Train Loss:  0.17302                 | Train Acc 1 min:  0.47792                 | Train Acc 5 min:  0.47848                 | Train Acc 10 min:  0.49264                 | Train Acc 15 min:  0.48754                 | Train Acc 30 min:  0.48188                 | Train Acc 1 hour:  0.49604                 | Train Acc 1 day:  0.48641                 | Val Loss:  0.17163                 | Val Acc 1 min:  0.40045                 | Val Acc 5 min:  0.37104                 | Val Acc 10 min:  0.38462                 | Val Acc 15 min:  0.38009                 | Val Acc 30 min:  0.41629                 | Val Acc 1 hour:  0.40498                 | Val Acc 1 day:  0.47285


In [16]:
torch.save(model.state_dict(), f'/content/drive/MyDrive/Диссертация/sbert_tg_{company}.pth')

### Маленькие Bert'ы

In [6]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1")

In [7]:
class BertClassifierTiny(nn.Module):

    def __init__(self, n_classes=7):

        super(BertClassifierTiny, self).__init__()

        self.bert = AutoModel.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1")
        self.fc1 = nn.Linear(264*2, 132)
        self.fc2 = nn.Linear(264*2+132, 264)
        self.fc3 = nn.Linear(264, n_classes)

        self.drop1 = nn.Dropout(0.1)
        self.drop2 = nn.Dropout(0.1)

        self.act = nn.ReLU()

    def forward(self, input_id, mask):
        model_output = self.bert(input_id, mask)
        sentence_embeddings = mean_pooling(model_output, mask)
        embeddings = torch.cat([model_output[0][:,0,:], sentence_embeddings], dim=1)
        embeddings1 = self.fc1(embeddings)
        embeddings = torch.cat([embeddings, embeddings1], dim=1)
        embeddings = self.act(embeddings)
        embeddings = self.drop1(embeddings)
        embeddings = self.fc2(embeddings)
        embeddings = self.act(embeddings)
        embeddings = self.drop2(embeddings)
        output = self.fc3(embeddings)

        return output

In [8]:
class TgDataset(Dataset):
    def __init__(self, texts, targets):
    
      self.labels = targets.astype(int)
      self.texts = [tokenizer(text, 
                          padding='max_length', max_length = 128, truncation=True,
                          return_tensors="pt") for text in texts]

    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):

        batch_texts = self.texts[idx]
        batch_y = self.labels[idx]

        return batch_texts, batch_y

In [30]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, device, epochs, scheduler=None):

    mean_1_min, mean_5_min, mean_10_min, mean_15_min, mean_30_min, mean_1_hour, mean_1_day = train_target.mean(axis=0)

    model = model.to(device)

    for epoch_num in range(epochs):
        model.train()

        acc_1_min_train = 0
        acc_5_min_train = 0
        acc_10_min_train = 0
        acc_15_min_train = 0
        acc_30_min_train = 0
        acc_1_hour_train = 0
        acc_1_day_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)

            input_id = train_input['input_ids'].squeeze(1).to(device)
            mask = train_input['attention_mask'].squeeze(1).to(device)

            output = model(input_id, mask)
            batch_loss = criterion(output, train_label.float())
            total_loss_train += batch_loss.item()
            out = torch.sigmoid(output)

            acc_1_min_train += ((out[:,0] > mean_1_min).int() == train_label[:,0]).sum().item()
            acc_5_min_train += ((out[:,1] > mean_5_min).int() == train_label[:,1]).sum().item()
            acc_10_min_train += ((out[:,2] > mean_10_min).int() == train_label[:,2]).sum().item()
            acc_15_min_train += ((out[:,3] > mean_15_min).int() == train_label[:,3]).sum().item()
            acc_30_min_train += ((out[:,4] > mean_30_min).int() == train_label[:,4]).sum().item()
            acc_1_hour_train += ((out[:,5] > mean_1_hour).int() == train_label[:,5]).sum().item()
            acc_1_day_train += ((out[:,6] > mean_1_day).int() == train_label[:,6]).sum().item()

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()
      
        acc_1_min_val = 0
        acc_5_min_val = 0
        acc_10_min_val = 0
        acc_15_min_val = 0
        acc_30_min_val = 0
        acc_1_hour_val = 0
        acc_1_day_val = 0
        total_loss_val = 0

        with torch.no_grad():
            model.eval()
            for val_input, val_label in tqdm(val_dataloader):

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.float())
                total_loss_val += batch_loss.item()
                out = torch.sigmoid(output)

                acc_1_min_val += ((out[:,0] > mean_1_min).int() == val_label[:,0]).sum().item()
                acc_5_min_val += ((out[:,1] > mean_5_min).int() == val_label[:,1]).sum().item()
                acc_10_min_val += ((out[:,2] > mean_10_min).int() == val_label[:,2]).sum().item()
                acc_15_min_val += ((out[:,3] > mean_15_min).int() == val_label[:,3]).sum().item()
                acc_30_min_val += ((out[:,4] > mean_30_min).int() == val_label[:,4]).sum().item()
                acc_1_hour_val += ((out[:,5] > mean_1_hour).int() == val_label[:,5]).sum().item()
                acc_1_day_val += ((out[:,6] > mean_1_day).int() == val_label[:,6]).sum().item()
      
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .5f} \
                | Train Acc 1 min: {acc_1_min_train / len(train_dataset): .5f} \
                | Train Acc 5 min: {acc_5_min_train / len(train_dataset): .5f} \
                | Train Acc 10 min: {acc_10_min_train / len(train_dataset): .5f} \
                | Train Acc 15 min: {acc_15_min_train / len(train_dataset): .5f} \
                | Train Acc 30 min: {acc_30_min_train / len(train_dataset): .5f} \
                | Train Acc 1 hour: {acc_1_hour_train / len(train_dataset): .5f} \
                | Train Acc 1 day: {acc_1_day_train / len(train_dataset): .5f} \
                | Val Loss: {total_loss_val / len(val_dataset): .5f} \
                | Val Acc 1 min: {acc_1_min_val / len(val_dataset): .5f} \
                | Val Acc 5 min: {acc_5_min_val / len(val_dataset): .5f} \
                | Val Acc 10 min: {acc_10_min_val / len(val_dataset): .5f} \
                | Val Acc 15 min: {acc_15_min_val / len(val_dataset): .5f} \
                | Val Acc 30 min: {acc_30_min_val / len(val_dataset): .5f} \
                | Val Acc 1 hour: {acc_1_hour_val / len(val_dataset): .5f} \
                | Val Acc 1 day: {acc_1_day_val / len(val_dataset): .5f}')

In [51]:
batch_size = 32
epochs = 5
train_dataset, val_dataset = TgDataset(train_texts, train_target), TgDataset(val_texts, val_target)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
total_steps = len(train_dataloader) * epochs

In [52]:
n_classes = 7
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BertClassifierTiny(n_classes)
LR = 1e-3
optimizer = AdamW(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(0.1*total_steps),
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational-v1 were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
model.bert.requires_grad = False

In [54]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, device, 1)

100%|██████████| 12/12 [00:00<00:00, 89.87it/s]

Epochs: 1 | Train Loss:  0.02205                 | Train Acc 1 min:  0.43182                 | Train Acc 5 min:  0.53237                 | Train Acc 10 min:  0.42287                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42218                 | Train Acc 1 hour:  0.44628                 | Train Acc 1 day:  0.47314                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


In [15]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Диссертация/tg_tiny_7_cls.pth')

In [56]:
model.bert.requires_grad = True

In [57]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, device, 15)

100%|██████████| 12/12 [00:00<00:00, 84.81it/s]


Epochs: 1 | Train Loss:  0.02205                 | Train Acc 1 min:  0.43664                 | Train Acc 5 min:  0.54132                 | Train Acc 10 min:  0.42493                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42218                 | Train Acc 1 hour:  0.44628                 | Train Acc 1 day:  0.47521                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


100%|██████████| 12/12 [00:00<00:00, 83.40it/s]


Epochs: 2 | Train Loss:  0.02202                 | Train Acc 1 min:  0.43388                 | Train Acc 5 min:  0.55028                 | Train Acc 10 min:  0.42424                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42355                 | Train Acc 1 hour:  0.44766                 | Train Acc 1 day:  0.46970                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


100%|██████████| 12/12 [00:00<00:00, 87.85it/s]


...

Epochs: 13 | Train Loss:  0.02202                 | Train Acc 1 min:  0.43182                 | Train Acc 5 min:  0.56474                 | Train Acc 10 min:  0.42424                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42355                 | Train Acc 1 hour:  0.45041                 | Train Acc 1 day:  0.47727                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


100%|██████████| 12/12 [00:00<00:00, 88.24it/s]


Epochs: 14 | Train Loss:  0.02204                 | Train Acc 1 min:  0.42975                 | Train Acc 5 min:  0.54959                 | Train Acc 10 min:  0.42355                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42355                 | Train Acc 1 hour:  0.44559                 | Train Acc 1 day:  0.47314                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


100%|██████████| 12/12 [00:00<00:00, 85.06it/s]

Epochs: 15 | Train Loss:  0.02205                 | Train Acc 1 min:  0.43388                 | Train Acc 5 min:  0.53168                 | Train Acc 10 min:  0.42355                 | Train Acc 15 min:  0.43388                 | Train Acc 30 min:  0.42287                 | Train Acc 1 hour:  0.44972                 | Train Acc 1 day:  0.47590                 | Val Loss:  0.02297                 | Val Acc 1 min:  0.35989                 | Val Acc 5 min:  0.62912                 | Val Acc 10 min:  0.34890                 | Val Acc 15 min:  0.33516                 | Val Acc 30 min:  0.35989                 | Val Acc 1 hour:  0.37363                 | Val Acc 1 day:  0.51099


In [ ]:
model()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/tg_sbert_7_cls.pth'))